In [6]:
import pandas as pd

# Load dataset
data = pd.read_csv("Sleep_dataset_transformed.csv")
data = data.drop(columns=["Person ID"])

# Shuffle the dataset for randomness
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into 80% training and 20% testing
split_index = int(0.8 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

# Feature columns and target column
columns = list(data.columns[:-1])  # All columns except the last one
target_col = "Sleep Disorder"

# Step 1: Get class-wise data and prior probabilities from training set
def get_class_data(train_data):
    class_data_dict = {}
    total = len(train_data)
    for label in train_data[target_col].unique():
        subset = train_data[train_data[target_col] == label]
        class_data_dict[label] = {
            "data": subset,
            "prior": len(subset) / total
        }
    return class_data_dict

# Step 2: Calculate conditional probability of input row given class
def calculate_prob(row, class_data, class_total):
    prob = 1
    for col in columns:
        val = row[col]
        match_count = len(class_data[class_data[col] == val])
        prob *= (match_count / class_total) if class_total > 0 else 0
    return prob

# Step 3: Predict class for a row
def predict(row, class_data_dict):
    max_prob = 0
    best_class = None
    for label, info in class_data_dict.items():
        class_data = info["data"]
        prior = info["prior"]
        class_total = len(class_data)
        likelihood = calculate_prob(row, class_data, class_total)
        posterior = likelihood * prior
        if posterior > max_prob:
            max_prob = posterior
            best_class = label
    return best_class

# Step 4: Test on test data and calculate accuracy
class_data_dict = get_class_data(train_data)

correct = 0
for index, row in test_data.iterrows():
    actual = row[target_col]
    predicted = predict(row, class_data_dict)
    if predicted == actual:
        correct += 1

accuracy = correct / len(test_data)
print(f"\nAccuracy on test set: {accuracy * 100:.2f}%")


Accuracy on test set: 89.33%


In [14]:
## import pandas as pd

# Load data
data = pd.read_csv("Sleep_dataset_transformed.csv")
data = data.drop(columns=["Person ID"])
columns = list(data.columns[:-1])

# Calculate total number of rows
total_rows = len(data)

# Split data by class
data_no = data[data["Sleep Disorder"] == "No Disorder"]  # Changed from "None" to "No Disorder"
data_apnea = data[data["Sleep Disorder"] == "Sleep Apnea"]
data_insomnia = data[data["Sleep Disorder"] == "Insomnia"]

# Calculate prior probabilities
p_no = len(data_no) / total_rows
p_apnea = len(data_apnea) / total_rows
p_insomnia = len(data_insomnia) / total_rows

# Collect input values
input_values = []
for col in columns:
    val = input(f"{col}: ")
    input_values.append(val)

# Function to calculate probability with Laplace smoothing
def calculate_prob(class_data, class_total):
    prob = 1
    for i in range(len(columns)):
        col = columns[i]
        val = input_values[i]
        # Add Laplace smoothing (add 1 to numerator and add number of possible values to denominator)
        match_count = len(class_data[class_data[col] == val]) + 1
        prob *= match_count / (class_total + len(data[col].unique()))
    return prob

# Calculate posterior probabilities
likelihood_no = calculate_prob(data_no, len(data_no)) * p_no
likelihood_apnea = calculate_prob(data_apnea, len(data_apnea)) * p_apnea
likelihood_insomnia = calculate_prob(data_insomnia, len(data_insomnia)) * p_insomnia

# Determine the prediction
if likelihood_no > likelihood_apnea and likelihood_no > likelihood_insomnia:
    prediction = "No Disorder"  # Changed from "None" to "No Disorder"
elif likelihood_apnea > likelihood_no and likelihood_apnea > likelihood_insomnia:
    prediction = "Sleep Apnea"
else:
    prediction = "Insomnia"

print("\nPredicted Sleep Disorder:", prediction)

Gender:  Male
Age:  Low
Occupation:  Docter
Sleep Duration:  Low
Quality of Sleep:  Bad
Physical Activity Level:  Moderate
Stress Level:  Medium
BMI Category:  Normal
Blood Pressure:  Mid
Heart Rate:  Mid
Daily Steps:  High



Predicted Sleep Disorder: Insomnia
